### Download Data 

* use coco dataset 2017 
* to save time, use validation dataset as training set 
* make folder structure to fulfill the bring your container folder structure 

In [ ]:
%%bash 
cd ~/SageMaker
mkdir input 
mkdir input/data 
cd input/data 
wget -O coco.zip https://tinyurl.com/yhtbxr6q 
unzip coco.zip 

### Build AlphaPose container 

* reference to [install instruction](https://github.com/catwhiskers/AlphaPose/blob/master/docs/INSTALL.md)
* fix package dependency pycocotools==2.0.2a1
* download pretrained models for inferences (yolov3 and fastpose)


In [ ]:
%%bash 
./build-and-push.sh 

### Do inference based on the docker image
* reference to the [inference instruction](https://github.com/catwhiskers/AlphaPose)

In [ ]:
!nvidia-docker run -it -v /home/ec2-user/SageMaker/:/opt/ml  --entrypoint='' 230755935769.dkr.ecr.us-west-2.amazonaws.com/alphapose-byos  python scripts/demo_inference.py --cfg configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml --checkpoint pretrained_models/fast_res50_256x192.pth --indir examples/demo/ --outdir /opt/ml/demo --save_img

### Training locally based on the docker image 

In [ ]:
!nvidia-docker run -it -v /home/ec2-user/SageMaker/:/opt/ml 230755935769.dkr.ecr.us-west-2.amazonaws.com/alphapose-byos 

### Use SageMaker Training Jobs 

In [ ]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = "alphapose"

#### upload data to s3 

In [ ]:
!cd ~/SageMaker/input/data/ && aws s3 cp --recursive coco s3://{bucket}/{prefix}/coco

#### define s3 input and output paths 

In [ ]:
coco_data = "s3://{}/{}/coco/".format(bucket, prefix)
outpath = "s3://{}/{}/output/".format(bucket, prefix)
repositoryUri = "230755935769.dkr.ecr.us-west-2.amazonaws.com/alphapose-byos"

#### define job_name and  and hyperparameters

In [ ]:
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
job_name = "alphapose-{}".format(str(int(timestamp))) 
job_name 

In [ ]:
hyperparameters = {
 "exp-id": "fastpose",
 "cfg": "/AlphaPose/configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml",
 "nThreads": 1
 }

In [ ]:
coco_input = sagemaker.inputs.TrainingInput(coco_data)

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                        role=role,
                        image_uri=repositoryUri,
                        source_dir='.',
                        instance_count=1,
                        instance_type='ml.p3.8xlarge',
                        framework_version='1.6.0',
                        py_version='py3',
                        sagemaker_session=sagemaker_session,
                        hyperparameters=hyperparameters, 
                        volume_size=100, 
                        debugger_hook_config=False
                   )


In [ ]:
estimator.fit(inputs={"coco":coco_input}, job_name=job_name)